In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from bs4 import Comment
import re

In [2]:
url = 'http://www.basketball-reference.com/leagues/NBA_2016.html'
data = requests.get(url).content
soup = BeautifulSoup(data, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [3]:
box_scores = []
for tag in soup.find_all("div", id='all_team-stats-per_game'):
    rows = []
    for i, row in enumerate(tag.find_all("tr")):
        if i==0:
            newtag="th"
        else:
            newtag="td"
        row_data = [item.get_text() for item in row.find_all(newtag)]
        rows.append(row_data)
    box_scores.append(rows)

In [4]:
print(tag.find_all("div"))

[<div class="section_heading">
<span class="section_anchor" data-label="Team Per Game Stats" id="team-stats-per_game_link"></span>
<h2>Team Per Game Stats</h2> <div class="section_heading_text">
<ul> <li><small>* Playoff teams</small></li>
</ul>
</div>
</div>, <div class="section_heading_text">
<ul> <li><small>* Playoff teams</small></li>
</ul>
</div>, <div class="placeholder"></div>]


In [5]:
html_data = tag.find_all(string=lambda text:isinstance(text,Comment))

In [30]:
headers = re.findall(string=html_data[0], pattern="<thead>.*</thead>", flags=re.DOTALL)[0]
data = html_data[0].split(headers)[1]
headers = re.findall(string=headers, pattern="<th.*</th>")
data = re.findall(string=data, pattern="<tr.*</tr>")
data, average_data = data[:-1], data[-1]
#Last element is the league average
pd_data = []
new_head = []
for header in headers:
    new_head.append(re.findall(string=header, pattern="\".*?\"")[0][1:-1])
new_data = []
for row in data:
    temp = list(filter(None, re.findall(string=row, pattern=r'>(.*?)<')))
    new_data.append([dummy for dummy in temp[1:] if dummy != '*'])

In [32]:
new_data = np.array(new_data)
table = pd.DataFrame(new_data, columns=new_head[1:])
print(table)

                      Team Games Minutes Played Field Goals  \
0    Golden State Warriors    82          242.4        42.5   
1    Oklahoma City Thunder    82          241.8        41.1   
2         Sacramento Kings    82          241.5        40.0   
3          Houston Rockets    82          241.8        37.7   
4           Boston Celtics    82          241.2        39.2   
5   Portland Trail Blazers    82          241.5        38.6   
6     Los Angeles Clippers    82          241.8        38.3   
7      Cleveland Cavaliers    82          242.1        38.7   
8       Washington Wizards    82          240.9        39.5   
9        San Antonio Spurs    82          240.3        40.1   
10       Charlotte Hornets    82          242.1        37.0   
11           Atlanta Hawks    82          241.8        38.6   
12    New Orleans Pelicans    82          241.2        38.5   
13         Toronto Raptors    82          241.2        36.7   
14  Minnesota Timberwolves    82          242.4        

In [7]:
url = 'http://www.basketball-reference.com/leagues/NBA_2017_per_game.html'
data = requests.get(url).content
soup = BeautifulSoup(data, 'html.parser')
data = soup.find_all("div", id='all_per_game_stats')[0]

In [23]:
table_stuff = data.find_all("table", id='per_game_stats')[0]
table_head = table_stuff.find('thead')
rows = table_head.find_all('tr')
for row in rows:
    elems = row.find_all('th')
    for elem in elems:
        curr_elem = elem.find(text=True)
        print(curr_elem)

Rk
Player
Pos
Age
Tm
G
GS
MP
FG
FGA
FG%
3P
3PA
3P%
2P
2PA
2P%
eFG%
FT
FTA
FT%
ORB
DRB
TRB
AST
STL
BLK
TOV
PF
PS/G


In [29]:
table_body = table_stuff.find_all("tbody")
for table_elem in table_body:
    rows = table_elem.find_all('tr')
    for row in rows:
        elems = row.find_all('td')
        for elem in elems:
            curr_elem = elem.find(text=True)
            print(curr_elem)

Alex Abrines
SG
23
OKC
16
0
12.6
1.4
4.1
.354
0.8
2.8
.273
0.7
1.3
.524
.446
0.6
0.7
.909
0.1
0.8
0.9
0.4
0.3
0.1
0.8
0.9
4.3
Quincy Acy
PF
26
DAL
6
0
8.0
0.8
2.8
.294
0.2
1.3
.125
0.7
1.5
.444
.324
0.3
0.5
.667
0.3
1.0
1.3
0.0
0.0
0.0
0.3
1.5
2.2
Steven Adams
C
23
OKC
18
18
31.6
4.2
7.8
.543
0.0
0.0
None
4.2
7.8
.543
.543
2.2
2.7
.816
2.8
4.9
7.7
0.7
1.3
1.1
2.1
2.6
10.7
Arron Afflalo
SG
31
SAC
16
14
27.5
3.3
7.6
.430
0.8
2.3
.333
2.5
5.3
.471
.479
1.9
2.1
.939
0.3
2.3
2.5
1.3
0.3
0.1
0.3
1.9
9.2
Alexis Ajinca
C
28
NOP
7
0
13.4
1.7
3.9
.444
0.0
0.0
None
1.7
3.9
.444
.444
0.4
0.6
.750
1.1
2.6
3.7
0.1
0.1
0.7
0.7
1.9
3.9
Cole Aldrich
C
28
MIN
16
0
12.3
1.1
2.1
.515
0.0
0.0
None
1.1
2.1
.515
.515
0.4
0.7
.636
1.1
2.5
3.6
0.4
0.6
0.4
0.3
2.4
2.6
LaMarcus Aldridge
PF
31
SAS
15
15
32.9
6.9
14.7
.473
0.5
1.1
.471
6.4
13.5
.473
.491
4.0
4.7
.857
1.8
4.9
6.7
1.5
0.8
1.1
1.5
2.2
18.4
Lavoy Allen
PF
27
IND
17
0
15.3
0.9
2.8
.340
0.0
0.1
.000
0.9
2.7
.348
.340
0.1
0.2
.500
1.4
1.8
3.1
1.0
0.3
0.3